 # Material de Referencia

- Transfer Learning with Transform by HuggingFace : https://notebook.community/deepchem/deepchem/examples/tutorials/22_Transfer_Learning_With_HuggingFace_tox21
- Simple Transformes:  https://simpletransformers.ai/docs/regression/
- https://simpletransformers.ai/docs/classification-models/#classificationmodel


### Utilizando Directamente BertForSequenceClassification

- https://stackoverflow.com/questions/63201036/add-additional-layers-to-the-huggingface-transformers
- https://huggingface.co/transformers/model_doc/bert.html
- https://huggingface.co/seyonec/PubChem10M_SMILES_BPE_450k


In [ ]:
!pip install transformers==3.3.0
!pip install simpletransformers

     |████████████████████████████████| 1.1 MB 4.1 MB/s 
     |████████████████████████████████| 3.0 MB 36.3 MB/s 
     |████████████████████████████████| 895 kB 52.9 MB/s 
     |████████████████████████████████| 1.2 MB 36.0 MB/s 
     |████████████████████████████████| 221 kB 4.3 MB/s 
     |████████████████████████████████| 264 kB 40.6 MB/s 
     |████████████████████████████████| 2.6 MB 36.7 MB/s 
     |████████████████████████████████| 124 kB 53.7 MB/s 
     |████████████████████████████████| 8.0 MB 36.9 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 1.6 MB 42.7 MB/s 
     |████████████████████████████████| 636 kB 39.0 MB/s 
     |████████████████████████████████| 3.3 MB 40.6 MB/s 
     |████████████████████████████████| 170 kB 46.1 MB/s 
     |████████████████████████████████| 97 kB 6.4 MB/s 
     |████████████████████████████████| 133 kB 49.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████

In [ ]:
%load_ext autoreload
%autoreload 2
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from transformers import TFBertForSequenceClassification, BertForSequenceClassification
import logging
import os.path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
path = ''
if not os.path.isfile('001_Data_retrieve.ipynb'):
  from google.colab import drive
  drive.mount('/content/drive')
  path = '/content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/TP-FINAL/bioinformatics_final_project/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWjWyftpiAj5qScWWTOkJ4jCLvU58Qq_clAiyExZUC7nVI1SopUb8bQ


## Cargar Datos y Split

In [ ]:
df = pd.read_csv(path+'data/acetylcholinesterase_02_bioactivity_data_preprocessed.csv')
max_len_idx = df['canonical_smiles'].apply(len).argmax()
min_len_idx = df['canonical_smiles'].apply(len).argmin()
max_sequence_len = len(df['canonical_smiles'].iloc[max_len_idx]) + 20

X = df['canonical_smiles'].values
y = df['pIC50'].values

#Pasar el input y output con el formato correcto
xy_df= df
xy_df = xy_df.drop(['molecule_chembl_id', 'standard_value',
       'standard_value_norm'],  axis=1).reset_index(drop=True)
train_df, eval_df = train_test_split(xy_df, test_size=0.2, random_state=42)

In [ ]:
train_df.head()

## MODELO 

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [14]:
# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 5
model_args.regression = True
model_args.overwrite_output_dir= True

#Early Stop
model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.01
model_args.early_stopping_metric = "r2_score"
model_args.early_stopping_metric_minimize = False
model_args.early_stopping_patience = 5

#model_args.evaluate_during_training_steps = 1000
#model_args.use_multiprocessing = True
#model_args.evaluate_during_training = True

model_args.tensorboard_dir= path+'tensorboard_fit'
model_args.save_model_every_epoch = False
model_args.save_eval_checkpoints = False
model_args.best_model_dir = path+'berta_best_model'

In [15]:
#model = ClassificationModel('roberta', 'seyonec/ChemBERTa-zinc-base-v1', use_cuda=False, args=model_args) # You can set class weights by using the optional weight argument
#'seyonec/PubChem10M_SMILES_BPE_450k'
#'seyonec/ChemBERTa-zinc-base-v1'
model = ClassificationModel('roberta','seyonec/PubChem10M_SMILES_BPE_450k', num_labels=1, args=model_args, use_cuda=True)

INFO:filelock:Lock 140494031587344 acquired on /root/.cache/huggingface/transformers/253794350adc4a34a5fa2c377562a56e4342cc22c69b456ef05893a09075094d.6662bce220e70bb69e1cc10c236b68e778001c010a6880b624c2159a235be52d.lock


Downloading:   0%|          | 0.00/515 [00:00<?, ?B/s]

INFO:filelock:Lock 140494031587344 released on /root/.cache/huggingface/transformers/253794350adc4a34a5fa2c377562a56e4342cc22c69b456ef05893a09075094d.6662bce220e70bb69e1cc10c236b68e778001c010a6880b624c2159a235be52d.lock
INFO:filelock:Lock 140493949350992 acquired on /root/.cache/huggingface/transformers/4157d41f0dbc8bcb88775edc1d4a8e7d31624059d3dbc8fcbec0284412d987b4.1295820a5c308a8f2816f86ff174331fd5cc41c1d01823fbbe83616c3b3bce2f.lock


Downloading:   0%|          | 0.00/336M [00:00<?, ?B/s]

INFO:filelock:Lock 140493949350992 released on /root/.cache/huggingface/transformers/4157d41f0dbc8bcb88775edc1d4a8e7d31624059d3dbc8fcbec0284412d987b4.1295820a5c308a8f2816f86ff174331fd5cc41c1d01823fbbe83616c3b3bce2f.lock
Some weights of the model checkpoint at seyonec/PubChem10M_SMILES_BPE_450k were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSe

...

INFO:filelock:Lock 140493976692496 released on /root/.cache/huggingface/transformers/25d3ef6d01385993df3adb1b86ac5f043ee4e705bab295d383578b76e8560de7.07b622242dcc6c7fd6a5f356d2e200c4f44be0279b767b85afcb24e778809d3c.lock
INFO:filelock:Lock 140493947452560 acquired on /root/.cache/huggingface/transformers/91488c95581c8d5e658a56e21f97ea0dabb3bc72e37b3a8fea1ffdbec3d56405.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock


Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

INFO:filelock:Lock 140493947452560 released on /root/.cache/huggingface/transformers/91488c95581c8d5e658a56e21f97ea0dabb3bc72e37b3a8fea1ffdbec3d56405.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock
INFO:filelock:Lock 140493947876496 acquired on /root/.cache/huggingface/transformers/e851dc8626f04c2f23abe880f881883012b0321e77574fc777942b12350e88ec.1788df22ba1a6817edb607a56efa931ee13ebad3b3500e58029a8f4e6d799a29.lock


Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

INFO:filelock:Lock 140493947876496 released on /root/.cache/huggingface/transformers/e851dc8626f04c2f23abe880f881883012b0321e77574fc777942b12350e88ec.1788df22ba1a6817edb607a56efa931ee13ebad3b3500e58029a8f4e6d799a29.lock


In [18]:
model.model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [16]:
#Entreno el modelo
model.train_model(train_df, eval_df=eval_df, overwrite_output_dir=True, acc=metrics.r2_score)


/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:616: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/4082 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_1_2


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/511 [00:00<?, ?it/s]

...

Running Epoch 2 of 5:   0%|          | 0/511 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/511 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/511 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


(2555, 1.448207127566061)

In [17]:
#Evaluacion del Modelo
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=metrics.r2_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1402: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1021 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_1_2


Running Evaluation:   0%|          | 0/128 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'acc': 0.6631669653838733, 'eval_loss': 0.9733124345075339}


### Comprobacion de metrica 

In [11]:
import numpy as np
#Mertrica del Problema
def R2(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def R2_numpy(y_true, y_pred):
    SS_res =  np.sum(np.square( y_true-y_pred ))
    SS_tot = np.sum(np.square( y_true - np.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + np.finfo(float).eps ) )

In [12]:
y_label = model_outputs.reshape(-1)
y_true = eval_df.pIC50.to_numpy()
y_true.shape, y_label.shape

((1021,), (1021,))

In [13]:
R2_numpy(y_true,model_outputs )

0.6743136425959146

## Simple-Viewer (Visualizing Model Predictions with Streamlit)[Permalink](https://simpletransformers.ai/docs/tips-and-tricks/)

## Hyperparameter Optimization https://wandb.ai/site/sweeps

## FIN